In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'path_to_your_excel_file.xlsx'
sheet_name = 'Montreal_CMA'

# Read the specific sheet
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the date column to datetime format if it's not already
df['date'] = pd.to_datetime(df['date'])

# Filter the date range
start_date = '2015-07-01'
end_date = '2024-07-31'
mask = (df['date'] >= start_date) & (df['date'] <= end_date)
filtered_df = df.loc[mask]

# Select the composite_HPI column
result = filtered_df[['date', 'composite_HPI']]

# Display the result
print(result)